In [34]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd

In [35]:
!ls va\ cong\ 2016

Virginia_Elections_Database__2016_U_S_House_General_Election_District_10_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_11_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_1_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_2_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_3_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_4_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_5_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_6_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_7_including_precincts.csv
Virginia_Elections_Database__2016_U_S_House_General_Election_District_8_including_precincts.csv
Virginia_Elections_Database__2016_U_S_

In [36]:
c = ['County/City',
 'Pct',
 'Republican',
 'Democratic',
 'All Others',
 'Total Votes Cast',
 'Independent',
 'Libertarian',
 'Green']

In [37]:
dfs = []
for i in range(1, 11):
    path = "va cong 2016/"
    f1 = "Virginia_Elections_Database__2016_U_S_House_General_Election_District_"
    f2 = "_including_precincts.csv"
    dfs.append(pd.read_csv(path+f1+str(i)+f2))

In [38]:
pres = pd.read_csv("../../presidential_2016/VA_Presidential_2016.csv")

In [39]:
pres.head(1)

,County/City,Ward,Pct,Hillary R. Clinton,Donald J. Trump,Gary Johnson,Evan McMullin,Jill Stein,All Others,Total Votes Cast
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
c = list(pres.columns)
c.remove("County/City")
c.remove("Pct")
pres.drop(columns=c, axis=1, inplace=True)
pres.head(2)

,County/City,Pct
0,NaN,NaN
1,Accomack County,101 - Chincoteague


In [41]:
pres.drop(index=0, inplace=True)
merged = pres.copy()
pres.head(1)

,County/City,Pct
1,Accomack County,101 - Chincoteague


In [42]:
print(len(pres))

2748


In [43]:
import difflib

def transform(s):
    if s == "None":
        return ""
    
    if "-" not in s:
        return s
    
    s = s.split("-")
    s = [a.strip() for a in s]
    s[0], s[1] = s[1], s[0]
    return '-'.join(s)
    
def findSimilar(s1, lst):
    max_ = 0
    word = ""
    s1 = transform(s1)
    if s1 == "None":
        return "None"
    
    og_s = ""
    for s in lst:
        og_s = s
        s = transform(s)
        r = difflib.SequenceMatcher(None, s1, s).ratio()
        if r > max_:
            max_ = r
            word = og_s
            
    return word

def createCountyPct(df):
    m = {}
    for index, row in df.iterrows():
        county = row["County/City"]
        if county not in m.keys():
            m[county] = [row["Pct"]]
        else:
            m[county].append(row["Pct"])
        #print(county)
    return m

def createRename(df):
    rename = {}
    c = list(df.columns)
    c.remove("County/City")
    c.remove("Ward")
    c.remove("Pct")
    c.remove("All Others")
    c.remove("Total Votes Cast")
    
    for cc in c:
        rename[cc] = df.iloc[0][cc]
    return rename

def addColumns(df):
    all_c = ["Republican", "Democratic", "Independent", "Libertarian", "Green", "All Others", "Total Votes Cast"]
    c = list(df.columns)
    
    for cc in all_c:
        if cc not in c:
            df[cc] = 0
            
def string2int(df):
    c = list(df.columns)
    c.remove("County/City")
    c.remove("Pct")
    c.remove("Ward")
    
    for cc in c:
        df = df.astype({cc: 'float64'})
    return df

In [44]:
addColumns(merged)
merged["dist_num"] = -1

# map district by district

In [45]:
def getDatas(row):
    mapper = {"Republican":row["Republican"], "Democratic":row["Democratic"], 
              "Independent":row["Independent"], "Libertarian": row["Libertarian"],
             "All Others":row["All Others"], "Total Votes Cast":row["Total Votes Cast"]}
    return mapper

def mergeDFs(df1, df2, dist_num):
    for index, row in df2.iterrows():
        mapper = getDatas(row)
        
        i = df1[(df1[("County/City")]==row["County/City"]) & (df1["Pct"]==row["Pct"])].index.to_list()
        if len(i) == 1:
            for key, item in mapper.items():
                if type(item) == str:
                    item = int(item.replace(",", ""))
                #print("\n", i)
                #print(df1.iloc[i[0]]["Pct"])
                df1.at[i, key] = (df1.loc[i[0]][key] + item)
            df1.at[i, "dist_num"] = dist_num
        else:
            print(i)
            print("error: mismatch", row["County/City"], row["Pct"])
    return df1

def mapp(index, merged, dist_num):
    cur_df = dfs[index]
    #cur_df = string2int(cur_df)

    rename = createRename(cur_df)

    cur_df.drop(index=0, inplace=True)
    cur_df.drop(columns=["Ward"], inplace=True, axis=1)

    index = cur_df[cur_df["County/City"] == "TOTALS"].index.to_list()[0]
    cur_df.drop(index=index, inplace=True, axis=0)

    if cur_df.isna().sum().sum():
        print("Has null")
        return ""

    # start matching
    missing = createCountyPct(cur_df)
    pres_name = createCountyPct(pres)

    matches = {}
    for county in missing.keys():
        matches[county] = {}
        precincts = missing[county]
        for pct in precincts:
            best = findSimilar(pct, pres_name[county])
            matches[county][pct] = best
            #print(pct, best)

    # mapping the matches
    for index, row in cur_df.iterrows():
        cur_df.at[index, "Pct"] = matches[row["County/City"]][row["Pct"]]

    cur_df.rename(mapper=rename, axis=1, inplace=True)
    addColumns(cur_df)

    # start mapping to pres
    merged = mergeDFs(merged, cur_df, dist_num)
    print(len(merged), len(pres))

    print("\nMerge:\n",merged.isna().sum())
    
    if len(merged) != len(pres):
        print("\ndrop duplicate")
        merged.drop_duplicates(subset=["Pct", "County/City"], inplace=True, keep="first")
    
        print("\nMerge Fixed:\n",merged.isna().sum())
    
    return merged

In [46]:
merged = mapp(0, merged, 1) 

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [47]:
merged = mapp(1, merged, 2)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [48]:
merged = mapp(2, merged, 3)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [49]:
merged = mapp(3, merged, 4)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [50]:
merged = mapp(4, merged, 5)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [51]:
merged = mapp(5,merged, 6)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [52]:
merged = mapp(6, merged, 7)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [53]:
merged = mapp(7, merged, 8)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [54]:
merged = mapp(8, merged, 9)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [55]:
merged = mapp(9, merged, 10)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [56]:
merged.head()

,County/City,Pct,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num
1,Accomack County,101 - Chincoteague,1220,397,0,0,0,2.0,1619,2
2,Accomack County,201 - Atlantic,595,126,0,0,0,2.0,723,2
3,Accomack County,202 - Greenbackville,739,261,0,0,0,1.0,1001,2
4,Accomack County,301 - New Church,620,634,0,0,0,0.0,1254,2
5,Accomack County,401 - Bloxom,366,109,0,0,0,0.0,475,2


In [57]:
len(merged)

2748

In [58]:
len(pres)

2748

In [59]:
merged["Type"] = "CONG2016"

In [60]:
merged.to_csv("VA_Congressional_2016_Cleand.csv", index=False)

# congressional 2018

In [61]:
dfs = []
for i in range(1, 11):
    path = "../congressional 2018/va cong 2018/"
    f1 = "Virginia_Elections_Database__2018_U_S_House_General_Election_District_"
    f2 = "_including_precincts.csv"
    dfs.append(pd.read_csv(path+f1+str(i)+f2))

In [62]:
pres = pd.read_csv("../../presidential_2016/VA_Presidential_2016.csv")

In [63]:
pres.head(1)

,County/City,Ward,Pct,Hillary R. Clinton,Donald J. Trump,Gary Johnson,Evan McMullin,Jill Stein,All Others,Total Votes Cast
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
c = list(pres.columns)
c.remove("County/City")
c.remove("Pct")
pres.drop(columns=c, axis=1, inplace=True)
pres.head(2)

,County/City,Pct
0,NaN,NaN
1,Accomack County,101 - Chincoteague


In [65]:
pres.drop(index=0, inplace=True)
merged = pres.copy()
pres.head(1)

,County/City,Pct
1,Accomack County,101 - Chincoteague


In [66]:
import difflib

def transform(s):
    if s == "None":
        return ""
    
    if "-" not in s:
        return s
    
    s = s.split("-")
    s = [a.strip() for a in s]
    s[0], s[1] = s[1], s[0]
    return '-'.join(s)
    
def findSimilar(s1, lst):
    max_ = 0
    word = ""
    s1 = transform(s1)
    if s1 == "None":
        return "None"
    
    og_s = ""
    for s in lst:
        og_s = s
        s = transform(s)
        r = difflib.SequenceMatcher(None, s1, s).ratio()
        if r > max_:
            max_ = r
            word = og_s
            
    return word

def createCountyPct(df):
    m = {}
    for index, row in df.iterrows():
        county = row["County/City"]
        if county not in m.keys():
            m[county] = [row["Pct"]]
        else:
            m[county].append(row["Pct"])
        #print(county)
    return m

def createRename(df):
    rename = {}
    c = list(df.columns)
    c.remove("County/City")
    c.remove("Ward")
    c.remove("Pct")
    c.remove("All Others")
    c.remove("Total Votes Cast")
    
    for cc in c:
        rename[cc] = df.iloc[0][cc]
    return rename

def addColumns(df):
    all_c = ["Republican", "Democratic", "Independent", "Libertarian", "Green", "All Others", "Total Votes Cast"]
    c = list(df.columns)
    
    for cc in all_c:
        if cc not in c:
            df[cc] = 0
            
def string2int(df):
    c = list(df.columns)
    c.remove("County/City")
    c.remove("Pct")
    c.remove("Ward")
    
    for cc in c:
        df = df.astype({cc: 'float64'})
    return df

In [67]:
addColumns(merged)
merged["dist_num"] = -1

# map district by district

In [68]:
def getDatas(row):
    mapper = {"Republican":row["Republican"], "Democratic":row["Democratic"], 
              "Independent":row["Independent"], "Libertarian": row["Libertarian"],
             "All Others":row["All Others"], "Total Votes Cast":row["Total Votes Cast"]}
    return mapper

def mergeDFs(df1, df2, dist_num):
    for index, row in df2.iterrows():
        mapper = getDatas(row)
        
        i = df1[(df1[("County/City")]==row["County/City"]) & (df1["Pct"]==row["Pct"])].index.to_list()
        if len(i) == 1:
            for key, item in mapper.items():
                if type(item) == str:
                    item = int(item.replace(",", ""))
                #print("\n", i)
                #print(df1.iloc[i[0]]["Pct"])
                df1.at[i, key] = (df1.loc[i[0]][key] + item)
            df1.at[i, "dist_num"] = dist_num
        else:
            print(i)
            print("error: mismatch", row["County/City"], row["Pct"])
    return df1

def mapp(index, merged, dist_num):
    cur_df = dfs[index]
    #cur_df = string2int(cur_df)

    rename = createRename(cur_df)

    cur_df.drop(index=0, inplace=True)
    cur_df.drop(columns=["Ward"], inplace=True, axis=1)

    index = cur_df[cur_df["County/City"] == "TOTALS"].index.to_list()[0]
    cur_df.drop(index=index, inplace=True, axis=0)

    if cur_df.isna().sum().sum():
        print("Has null")
        return ""

    # start matching
    missing = createCountyPct(cur_df)
    pres_name = createCountyPct(pres)

    matches = {}
    for county in missing.keys():
        matches[county] = {}
        precincts = missing[county]
        for pct in precincts:
            best = findSimilar(pct, pres_name[county])
            matches[county][pct] = best
            #print(pct, best)

    # mapping the matches
    for index, row in cur_df.iterrows():
        cur_df.at[index, "Pct"] = matches[row["County/City"]][row["Pct"]]

    cur_df.rename(mapper=rename, axis=1, inplace=True)
    addColumns(cur_df)

    # start mapping to pres
    merged = mergeDFs(merged, cur_df, dist_num)
    print(len(merged), len(pres))

    print("\nMerge:\n",merged.isna().sum())
    
    if len(merged) != len(pres):
        print("\ndrop duplicate")
        merged.drop_duplicates(subset=["Pct", "County/City"], inplace=True, keep="first")
    
        print("\nMerge Fixed:\n",merged.isna().sum())
    
    return merged

In [69]:
merged = mapp(0, merged, 1) 

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [70]:
merged = mapp(1, merged, 2)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [71]:
merged = mapp(2, merged, 3)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [72]:
merged = mapp(3, merged, 4)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [73]:
merged = mapp(4, merged, 5)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [74]:
merged = mapp(5,merged, 6)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [75]:
merged = mapp(6, merged, 7)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [76]:
merged = mapp(7, merged, 8)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [77]:
merged = mapp(8, merged, 9)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [78]:
merged = mapp(9, merged, 10)

2748 2748

Merge:
 County/City         0
Pct                 0
Republican          0
Democratic          0
Independent         0
Libertarian         0
Green               0
All Others          0
Total Votes Cast    0
dist_num            0
dtype: int64


In [79]:
merged.head()

,County/City,Pct,Republican,Democratic,Independent,Libertarian,Green,All Others,Total Votes Cast,dist_num
1,Accomack County,101 - Chincoteague,974,465,0,0,0,1.0,1440,2
2,Accomack County,201 - Atlantic,452,149,0,0,0,2.0,603,2
3,Accomack County,202 - Greenbackville,558,268,0,0,0,1.0,827,2
4,Accomack County,301 - New Church,459,508,0,0,0,0.0,967,2
5,Accomack County,401 - Bloxom,268,103,0,0,0,0.0,371,2


In [80]:
len(merged)

2748

In [81]:
len(pres)

2748

In [82]:
merged["Type"] = "CONG2018"

In [83]:
merged.to_csv("../congressional 2018/VA_Congressional_2018_Cleand.csv", index=False)